In [4]:
from argparse import Namespace

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, PreTrainedModel, GenerationConfig, PreTrainedTokenizer
from peft import LoraConfig, get_peft_model, PeftModel
from torch.utils.data import Dataset, DataLoader, DistributedSampler


from openreviewer.arguments import get_args
from openreviewer.dataset import InstructionTuningDataset
from openreviewer.utils import vicuna_sample_processor, print_rank, broadcast_model, move_dict_to_device, save_checkpoint, openreviewer_data_preprocessor
from openreviewer.scheduler import CosineWarmUpScheduler
from openreviewer.common import freeze_ffn_target_moudles, lora_target_modules

from main import get_dataset

model_path = "/root/autodl-tmp/model/vicuna-7b-v1.5-16k"
save_path = "/root/autodl-tmp/checkpoints/1204"

tokenizer = AutoTokenizer.from_pretrained(model_path)
args = Namespace(
    dataset_type="InstructionTuningDataset",
    data_path="/root/autodl-tmp/data/iclr2024/1204.jsonl",
    model_type="vicuna",
    max_length=6144,
    max_prompt_length=4096,
)

dataset = get_dataset(args, tokenizer, preprocessor=openreviewer_data_preprocessor)

print(dataset[0])

model = AutoModelForCausalLM.from_pretrained(model_path)
model = PeftModel.from_pretrained(model, save_path)


{'system_message': 'You are reviewing the paper titled Learning SO(3)-Invariant Correspondence via Point-wise Local Shape Transform. The keywords are Point cloud understanding, 3D dense correspondence, SO(3)-invariance, Part label transfer. You will read this paper and write a review for it.', 'prompt': 'The abstract is：\n\nEstablishing accurate dense 3D correspondences between diverse shapes stands as a pivotal challenge with profound implications for computer vision and robotics. However, existing self-supervised methods for this problem assume perfect input shape alignment, restricting their real-world applicability. In this work, we introduce a novel self-supervised SO(3)-invariant 3D correspondence learner, dubbed LSTNet, that learns to establish dense correspondences between shapes even under challenging intra-class variations. Specifically, LSTNet learns to dynamically formulate an SO(3)-invariant local shape transform for each point, which maps the SO(3)-equivariant global shap

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
generation_config = GenerationConfig(
    do_sample=True,
    temerature=0.7,
    max_new_tokens=1024
)

args.batch_size = 1
args.num_workers = 1

dp_rank=0
dp_world_size = 1

sampler = DistributedSampler(
    dataset,
    shuffle=False,
    drop_last=False,
    rank=dp_rank,
    num_replicas=dp_world_size
)

dataloader = DataLoader(
    dataset,
    sampler=sampler,
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    collate_fn=dataset.collate
)

model.eval()
model.cuda()

results = []

for num_samples, (input_batch, gen_batch, other_batch) in enumerate(dataloader):
    move_dict_to_device(gen_batch, model.device)
    output_ids = model.generate(**gen_batch, generation_config=generation_config)
    inputs = tokenizer.batch_decode(gen_batch['input_ids'])
    outputs = tokenizer.batch_decode(output_ids[:, gen_batch['input_ids'].size(-1):])
    for input_str, output_str in zip(inputs, outputs):
        print('===')
        print(input_str)
        print('---')
        print(output_str)
        print()
    results.append(
        {
            "prompt": input_str,
            "response": output_str
        }
    )
    if num_samples == 16:
        break

===
<s> You are reviewing the paper titled Learning SO(3)-Invariant Correspondence via Point-wise Local Shape Transform. The keywords are Point cloud understanding, 3D dense correspondence, SO(3)-invariance, Part label transfer. You will read this paper and write a review for it. USER: The abstract is：

Establishing accurate dense 3D correspondences between diverse shapes stands as a pivotal challenge with profound implications for computer vision and robotics. However, existing self-supervised methods for this problem assume perfect input shape alignment, restricting their real-world applicability. In this work, we introduce a novel self-supervised SO(3)-invariant 3D correspondence learner, dubbed LSTNet, that learns to establish dense correspondences between shapes even under challenging intra-class variations. Specifically, LSTNet learns to dynamically formulate an SO(3)-invariant local shape transform for each point, which maps the SO(3)-equivariant global shape descriptor of the i

In [15]:
import json

with open('results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)
